In [1]:
! pip install -q accelerate  peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 26.8 MB/s eta 0:00:00


In [2]:
import torch
import os
from datasets import load_dataset, Dataset
from google.colab import userdata
from transformers import (AutoModelForCausalLM,
AutoTokenizer,BitsAndBytesConfig,
HfArgumentParser, TrainingArguments,
logging, pipeline ,GemmaTokenizer)
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer , SFTConfig

In [3]:
os.environ['HF-token'] = userdata.get('HF-token')

In [4]:
model_id = "google/gemma-2b"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    )


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token = os.environ['HF-token'])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map={"":0},
    token = os.environ['HF-token']
    )

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [15]:
input_text = "یک شعر کوتاه درباره ماشین لرنینگ بنویس"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

<bos>یک شعر کوتاه درباره ماشین لرنینگ بنویسید.

<h2>پاسخ</h2>

1. ماشین لرنینگ یک ماشین است که در سال ۱۹


In [6]:
Lora_config = LoraConfig(
    r=16,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
    )

In [7]:
import pandas as pd
dds = load_dataset("kakooch/persian-poetry-qa")
ds = dds["train"]
ds = ds.to_pandas()
df_sampled = ds.sample(n=50000, random_state=42)
df = df_sampled[["question", "answer"]]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

poems-qa.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1366625 [00:00<?, ? examples/s]

In [ ]:

df.to_csv("drive/MyDrive/QLORA/50k_poem_sample.csv", index=False)

In [11]:
dataset = Dataset.from_pandas(df)

In [ ]:

dataset.push_to_hub("Erfan3940/50k_persian_poem_author",token = os.environ['HF-token'] )

In [10]:
df.head()

,question,answer
943845,یک نمونه از شعر فردوسی,برهنه کشانش ببر تا بچاه \t که در چاه بین آنک د...
401204,یک نمونه از شعر حکیم سبزواری,چو افتاده ما را که کام دگرها \t اگر از تو گاهی...
878491,یک نمونه از شعر غروی اصفهانی,ای ملامت گر از اسرار قدر بیخبری \t که دچار غم ...
667922,یک نمونه از شعر صائب تبریزی,زسختی قابل اصلاح نبود دل ترا، ورنه \t ازین دری...
745830,یک نمونه از شعر عبید زاکانی,چو من به فکر فرو رفته و روان کرده \t دبیر چرخ ...


In [29]:
def formatting (example):
    text = f"answer: {example['answer'][0]}\nquestion: {example['question'][0]}"
    return [text]

In [8]:
myds = load_dataset("Erfan3940/50k_persian_poem_author",token = os.environ['HF-token'])

README.md:   0%|          | 0.00/359 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.55M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
cleaned = myds.filter(
    lambda x: x["question"] is not None
              and x["answer"] is not None
              and str(x["question"]).strip() != ""
              and str(x["answer"]).strip() != ""
)
cleaned.save_to_disk("cleaned_dataset")

cleaned


Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/49748 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', '__index_level_0__'],
        num_rows: 49748
    })
})

In [54]:
def preprocess(example):
    poem = example["answer"]
    author = example["question"]

    text = f"question: {poem}\nanswer: {author}"

    token = tokenizer(
        text,
        max_length=512,
        truncation=True,
    )

    labels = token["input_ids"].copy()
    labels = [-100 if tok == tokenizer.pad_token_id else tok for tok in labels]
    token["labels"] = labels
    return token


tokenize_dataset = cleaned.map(preprocess, batched=False)



Map:   0%|          | 0/49748 [00:00<?, ? examples/s]

In [55]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenize_dataset['train'],
    peft_config=Lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="drive/MyDrive/QLORA/gemma3",
        optim="paged_adamw_8bit"
    ),
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/49748 [00:00<?, ? examples/s]

In [56]:
trainer.train()

Step,Training Loss
1,5.833300
2,5.553000
3,6.153800
4,5.406100
5,5.673700
6,5.474300
7,5.100000
8,5.041400
9,4.845500
10,4.654600


/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1394: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-692c72a5-72964d366f18f28570a42254;7ced2387-3a78-4681-aac8-fb24b6aff87f)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=100, training_loss=3.736827576160431, metrics={'train_runtime': 97.8094, 'train_samples_per_second': 4.09, 'train_steps_per_second': 1.022, 'total_flos': 174490426859520.0, 'train_loss': 3.736827576160431, 'epoch': 0.00804052424218059})

In [77]:
text = "question:ادلا این نور اندر آخر کار حجاب از پیش بردارد بیکبارد\nanswer:"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    # repetition_penalty=1.25,
    # no_repeat_ngram_size=3,
    # do_sample=True,

)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

question:ادلا این نور اندر آخر کار حجاب از پیش بردارد بیکبارد
answer: یک نمونه از شعر عطار نیشابوری


In [80]:
model.push_to_hub("Erfan3940/finetuned_Gemma3_on_50k_persian_poems",token=os.environ["HF-token"])
tokenizer.push_to_hub("Erfan3940/finetuned_Gemma3_on_50k_persian_poems",token=os.environ["HF-token"])

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...xc_3228/model.safetensors:   1%|1         | 23.3MB / 2.24GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...pl9ko_eid/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...mpl9ko_eid/tokenizer.json:  97%|#########7| 33.4MB / 34.4MB            

CommitInfo(commit_url='https://huggingface.co/Erfan3940/finetuned_Gemma3_on_50k_persian_poems/commit/024a4f0e224dd545bdc91486ad2974aef9ea0962', commit_message='Upload tokenizer', commit_description='', oid='024a4f0e224dd545bdc91486ad2974aef9ea0962', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Erfan3940/finetuned_Gemma3_on_50k_persian_poems', endpoint='https://huggingface.co', repo_type='model', repo_id='Erfan3940/finetuned_Gemma3_on_50k_persian_poems'), pr_revision=None, pr_num=None)

In [ ]:
# widjet deleter for github push
import json

with open('drive/MyDrive/Colab Notebooks/Gemma3_finetu.ipynb', 'r', encoding='utf-8') as f:
    notebook = json.load(f)

# Remove problematic widgets metadata
if 'metadata' in notebook and 'widgets' in notebook['metadata']:
    del notebook['metadata']['widgets']

# Or fix individual cells
for cell in notebook.get('cells', []):
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']
    # Also check for empty widget arrays
    if 'metadata' in cell and 'widgets' in cell['metadata'] and not cell['metadata']['widgets']:
        del cell['metadata']['widgets']

with open('drive/MyDrive/Colab Notebooks/Qlora1.ipynb', 'w', encoding='utf-8') as f:
    json.dump(notebook, f, indent=1)